# SDES Project 1 Animation
### Name: Soumallya Chatterjee
### Roll No: 153070018

# LC tank circuit with resistance
- LC tank circuit is nothing but the parrallel combination of inductor L and capacitor C and L and C has ESRs of $R_L$ and $R_c$
$$L \frac{di_L}{dt} + R_L i_L(t) = V$$ 
$$\frac{1}{C}\int_{0}^{t} i_c dt + v_c(0) + R_c i_c (t) = V$$
$$i(t) = i_L (t) + i_c (t)$$
- This is an ani-resonating circuit with anti-resonating frequency $\omega_R$:
$$\omega_R = \frac{1}{\sqrt{LC}}$$
The above equation holds for ideal cases i.e. ESRs of both L acd C zero.
- But when ESRs are non-zero the above equation does not hold.

In [1]:
# First of all consider the ideal case and check the voltage and current waveform

from __future__ import division
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy

def voltage(A,w,t):
    return A*numpy.sin(w*t)

def voltage_derivative(A,w,t):
    return w*A*numpy.cos(w*t)

def response(A,w,L,R1,R2,C,t):

    def LC_tank_inductor(A,initial,t,w,L,R1):
        def tank_int1(i1,t):
            F1 = voltage(A,w,t)/L - (R1/L)*i1
            return F1
        i1 = odeint(tank_int1,initial,t)
        i1 = i1.flat
        return i1
   
    def LC_tank_capacitor(A,initial,t,w,C,R2):
        if R2==0:
            i2 = C*voltage_derivative(A,w,t)
            #for T in t:
            #   i2.append([C*voltage(A,w,t)])
            #i2 = numpy.array(i2)
            return i2
        else:
  
            def tank_int2(i2,t):
                F2 = voltage_derivative(A,w,t)/R2-i2/(R2*C)
                return F2
            i2 = odeint(tank_int2,initial,t)
            i2 = i2.flat
            return i2


    V = voltage(A,w,t)
    V = V.tolist()

    #def func(t):
    #    return numpy.sin(w*t)

    i1 = LC_tank_inductor(A,0,t,w,L,R1)
    #print "i1=",i1
    #print type(i1)
    i2 = LC_tank_capacitor(A,0,t,w,C,R2)
    #print "i2=",i2

    i = []
    for j in range(len(t)):
        i.append(i1[j] + i2[j])

    Voltage_difference = 2*A
    gain = abs(max(i)-min(i))/Voltage_difference
    #print "Voltage difference =", Voltage_difference

    return i,V,t,gain

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
plt.title('Voltage and current vs time Roll: 153070018')
plt.xlabel('Time in seconds')
plt.ylabel('Current in Ampere and Voltage in Volts')
ax = fig.add_subplot(111, autoscale_on=False, xlim=(0,5), ylim=(-5, 5))
line, = ax.plot([], [],'-', lw=2,label = 'Voltage')
line2, = ax.plot([], [],'.', lw=2,label = 'Current')
plt.legend(handles=[line, line2])

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    line2.set_data([], [])
    return line, line2,

A = 1
w_dash = 1
L_dash = 0.5
R1 = 0
R2 = 0
C_dash = 0.5

t = numpy.array(range(0,300,1))
t = t/10
i1,V,t,gain = response(A,w_dash,L_dash,R1,R2,C_dash,t)
# animation function.  This is called sequentially
def animate(i):
    line.set_data(t-0.1*i,V)
    line2.set_data(t-0.1*i,i1)
    return line,line2

# call the animator.  blit=True means only re-draw the parts that have changed.
ani = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit= False)

HTML(ani.to_html5_video())

- Now consider the case when both inductor and capacitor has resistance of 1 $\Omega$ and check the voltage and current waveforms.

In [2]:
A = 1
w_dash = 1
L_dash = 0.5
R1 = 1
R2 = 1
C_dash = 0.5

t = numpy.array(range(0,300,1))
t = t/10
i1,V,t,gain = response(A,w_dash,L_dash,R1,R2,C_dash,t)
# animation function.  This is called sequentially
def animate(i):
    line.set_data(t-0.1*i,V)
    line2.set_data(t-0.1*i,i1)
    return line,line2

# call the animator.  blit=True means only re-draw the parts that have changed.
ani = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit= False)

HTML(ani.to_html5_video())

- Now consider the case when ESRs of both inductor and capacitor us zero and obtain the frequency response

In [3]:
#Gain vs frequency response when both inductor resistance and capacitor resistance are zero
t = numpy.array(range(0,30000,1))
t = t/1000

L = 0.5
R11 = 0
R12 = 0
C = 0.5
A = 1

frequency = numpy.array(range(1,100))
frequency = frequency/10
Gain1 = []
for w in frequency:
    i,V,t,gain = response(A,w,L,R11,R12,C,t)
    #print "Frequency = {},Gain = {}".format(w,gain)
    Gain1.append(gain)
    
fig1 = plt.figure()
plt.title('Gain vs frequency Roll: 153070018')
plt.ylabel('Gain')
plt.xlabel('Frequency in rad/sec')
ax = fig1.add_subplot(111, autoscale_on=False, xlim=(0,10), ylim=(0, 20))
ax.grid()
line, = ax.plot([], [], '-', lw=2)

def init():
    line.set_data([], [])
    return line,

thisx = []
thisy = []
def animate(i):
    thisx.append(frequency[i])
    thisy.append(Gain1[i])

    line.set_data(thisx, thisy)
    return line,

ani1 = animation.FuncAnimation(fig1, animate, numpy.arange(1, len(frequency)),
                              interval=25, blit=False, init_func=init)
HTML(ani1.to_html5_video())

- Now consider the case where ESR of inductor is 1 $\Omega$ and ESR of capacitor is 0 and check the frequency response.

In [4]:
# Gain vs frequency curve when inductor resistance is 1 ohm and capacitor resistance is zero

R21 = 1
R22 = 0

frequency = numpy.array(range(1,100))
frequency = frequency/10
Gain2 = []
for w in frequency:
    i,V,t,gain = response(A,w,L,R21,R22,C,t)
    #print "Frequency = {},Gain = {}".format(w,gain)
    Gain2.append(gain)
    
fig2 = plt.figure()
plt.title('Gain vs frequency Roll: 153070018')
plt.ylabel('Gain')
plt.xlabel('Frequency in rad/sec ')
ax = fig2.add_subplot(111, autoscale_on=False, xlim=(0,10), ylim=(0, 6))
ax.grid()
line, = ax.plot([], [], '-', lw=2)

def init():
    line.set_data([], [])
    return line,

thisx = []
thisy = []
def animate(i):
    thisx.append(frequency[i])
    thisy.append(Gain2[i])

    line.set_data(thisx, thisy)
    return line,

ani2 = animation.FuncAnimation(fig2, animate, numpy.arange(1, len(frequency)),
                              interval=25, blit=False, init_func=init)
HTML(ani2.to_html5_video())

- Now consider the case where ESR of inductor is 0 and ESR of capacitor is 1 $\Omega$ and check the frequency response.

In [5]:
# Gain vs frequency curve when the capacitor resistance is 1 ohm and inductor resistance is zero

R31 = 0
R32 = 1


Gain3 = []
for w in frequency:
    i,V,t,gain = response(A,w,L,R31,R32,C,t)
    #print "Frequency = {},Gain = {}".format(w,gain)
    Gain3.append(gain)
    
fig3 = plt.figure()
plt.title('Gain vs frequency Roll: 153070018')
plt.ylabel('Gain')
plt.xlabel('Frequency in rad/sec')
ax = fig3.add_subplot(111, autoscale_on=False, xlim=(0,10), ylim=(0, 10))
ax.grid()
line, = ax.plot([], [], '-', lw=2)

def init():
    line.set_data([], [])
    return line,

thisx = []
thisy = []
def animate(i):
    thisx.append(frequency[i])
    thisy.append(Gain3[i])

    line.set_data(thisx, thisy)
    return line,

ani3 = animation.FuncAnimation(fig3, animate, numpy.arange(1, len(frequency)),
                              interval=25, blit=False, init_func=init)
HTML(ani3.to_html5_video())

- Now consider the case where ESRs of both inductor and capacitor is 1 $\Omega$.

In [7]:
#Gain vs frequency curve when both of the resistances value are 1 ohm

R41 = 1
R42 = 1


Gain4 = []
for w in frequency:
    i,V,t,gain = response(A,w,L,R41,R42,C,t)
    #print "Frequency = {},Gain = {}".format(w,gain)
    Gain4.append(gain)
    
fig4 = plt.figure()
plt.title('Gain vs frequency Roll: 153070018')
plt.ylabel('Gain')
plt.xlabel('Frequency in rad/sec')
ax = fig4.add_subplot(111, autoscale_on=False, xlim=(0,10), ylim=(0, 2))
ax.grid()
line, = ax.plot([], [], '-', lw=2)

def init():
    line.set_data([], [])
    return line,

thisx = []
thisy = []
def animate(i):
    thisx.append(frequency[i])
    thisy.append(Gain4[i])

    line.set_data(thisx, thisy)
    return line,

ani4 = animation.FuncAnimation(fig4, animate, numpy.arange(1, len(frequency)),
                              interval=25, blit=False, init_func=init)
HTML(ani4.to_html5_video())

- Now consider the case when ESR of the inductor is 1 $\Omega$ and ESR of the capacitor is 0.5 $\Omega$.

In [8]:
# Gain vs frequency curve when the inductor resistance is 1 ohm and capacitor resistance is 0.5 ohm

R51 = 1
R52 = 0.5


Gain5 = []
for w in frequency:
    i,V,t,gain = response(A,w,L,R51,R52,C,t)
    #print "Frequency = {},Gain = {}".format(w,gain)
    Gain5.append(gain)
    
fig5 = plt.figure()
plt.title('Gain vs frequency Roll: 153070018')
plt.ylabel('Gain')
plt.xlabel('Frequency in rad/sec')
ax = fig5.add_subplot(111, autoscale_on=False, xlim=(0,10), ylim=(0, 2))
ax.grid()
line, = ax.plot([], [], '-', lw=2)

def init():
    line.set_data([], [])
    return line,

thisx = []
thisy = []
def animate(i):
    thisx.append(frequency[i])
    thisy.append(Gain5[i])

    line.set_data(thisx, thisy)
    return line,

ani5 = animation.FuncAnimation(fig5, animate, numpy.arange(1, len(frequency)),
                              interval=25, blit=False, init_func=init)
HTML(ani5.to_html5_video())

- Now consider the case when ESR of inductor is 0.5 $\Omega$ and ESR of capacitance is 1 $\Omega$.

In [9]:
# Gain vs frequency curve when the inductor resistance is 0.5 ohm and capacitor resistance is 1 ohm

R61 = 0.5
R62 = 1


Gain6 = []
for w in frequency:
    i,V,t,gain = response(A,w,L,R61,R62,C,t)
    #print "Frequency = {},Gain = {}".format(w,gain)
    Gain6.append(gain)
    
fig6 = plt.figure()
plt.title('Gain vs frequency Roll: 153070018')
plt.ylabel('Gain')
plt.xlabel('Frequency in rad/sec')
ax = fig6.add_subplot(111, autoscale_on=False, xlim=(0,10), ylim=(0, 2))
ax.grid()
line, = ax.plot([], [], '-', lw=2)

def init():
    line.set_data([], [])
    return line,

thisx = []
thisy = []
def animate(i):
    thisx.append(frequency[i])
    thisy.append(Gain6[i])

    line.set_data(thisx, thisy)
    return line,

ani6 = animation.FuncAnimation(fig6, animate, numpy.arange(1, len(frequency)),
                              interval=25, blit=False, init_func=init)
HTML(ani6.to_html5_video())